# Running CEDS Scenarios

In this notebook we document how to process and run data from the CEDS database.

In [18]:
%matplotlib inline

from os import listdir
from os.path import join, dirname
from pprint import pprint

import pandas as pd
import pyam
import pymagicc
from pymagicc.io import MAGICCData
import matplotlib.pyplot as plt
plt.style.use('bmh') 

import expectexception

In [19]:
TEST_DATA_PATH = join("..", "tests", "test_data")

In [20]:
def read_ceds_csv(file_to_read):
    pyam_df = pyam.IamDataFrame(
        data=file_to_read,
        encoding="utf-8"
    )
    
    raw_df = pyam_df.data
    
    reindexed_df = raw_df.set_index(["model", "scenario", "region", "variable", "unit", "year"]).unstack().T
    reindexed_df.index = reindexed_df.index.get_level_values("year")
    reindexed_df.index.name = "YEAR"
    models = reindexed_df.columns.get_level_values("model")
    scenarios = reindexed_df.columns.get_level_values("scenario")
    regions = reindexed_df.columns.get_level_values("region")
    variables = reindexed_df.columns.get_level_values("variable")
    units = reindexed_df.columns.get_level_values("unit")
    todos = ["SET"] * len(units)
    reindexed_df.columns = pd.MultiIndex.from_arrays(
        [models, scenarios, variables, todos, units, regions],
        names=("MODEL", "SCENARIO", "VARIABLE", "TODO", "UNITS", "REGION"),
    )
    
    return reindexed_df

In [21]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [29]:
ceds_df = read_ceds_csv(join(TEST_DATA_PATH, "ceds-format-example.csv"))
groupby_and_drop_levels = ["MODEL", "SCENARIO"]
for a, b in ceds_df.groupby(axis=1, level=groupby_and_drop_levels):
    metadata = {level: a[i] for i, level in enumerate(groupby_and_drop_levels)}
    metadata
    b
    b.columns = b.columns.droplevel(level=groupby_and_drop_levels)
    b

INFO:root:Reading `../tests/test_data/ceds-format-example.csv`


{'MODEL': 'MODEL-NAME-HYPHENS', 'SCENARIO': 'SCENARIO-A-B-CDE-2'}

MODEL    MODEL-NAME-HYPHENS                                          \
SCENARIO SCENARIO-A-B-CDE-2                                           
VARIABLE       Emissions|BC Emissions|BC|Agricultural Waste Burning   
TODO                    SET                                     SET   
UNITS              Mt BC/yr                                Mt BC/yr   
REGION               R5ASIA                                  R5ASIA   
YEAR                                                                  
2015                      2                                       8   
2020                      7                                       1   
2030                      3                                       9   
2040                     10                                       8   
2050                      7                                       2   
2060                      0                                       9   
2070                      0                                       6   
2080                      2                                       1   
2090                     10                                       6   
2100                      8                                       8   

MODEL                                                            \
SCENARIO                                                          
VARIABLE Emissions|BC|Energy Sector Emissions|BC|Forest Burning   
TODO                            SET                         SET   
UNITS                      Mt BC/yr                    Mt BC/yr   
REGION                       R5ASIA                      R5ASIA   
YEAR                                                              
2015                              7                           5   
2020                              4                           2   
2030                              6                           1   
2040                              6                          10   
2050                              9                           8   
2060                              1                          10   
2070                              3                          10   
2080                              6                           0   
2090                              6                           4   
2100                              4                           4   

MODEL                                                                   \
SCENARIO                                                                 
VARIABLE Emissions|BC|Grassland Burning Emissions|BC|Industrial Sector   
TODO                                SET                            SET   
UNITS                          Mt BC/yr                       Mt BC/yr   
REGION                           R5ASIA                         R5ASIA   
YEAR                                                                     
2015                                  6                              1   
2020                                  7                              9   
2030                                  7                              4   
2040                                  4                              9   
2050                                  3                              0   
2060                                  2                              5   
2070                                  3                              0   
2080                                  4                              6   
2090                                  3                             10   
2100                                  1                              3   

MODEL                                                                         \
SCENARIO                                                                       
VARIABLE Emissions|BC|Peat Burning Emissions|BC|Residential Commercial Other   
TODO                           SET                                       SET   
UNITS                     Mt BC/yr                                  Mt BC

VARIABLE,Emissions|BC,Emissions|BC|Agricultural Waste Burning,Emissions|BC|Energy Sector,Emissions|BC|Forest Burning,Emissions|BC|Grassland Burning,Emissions|BC|Industrial Sector,Emissions|BC|Peat Burning,Emissions|BC|Residential Commercial Other,Emissions|BC|Transportation Sector,Emissions|BC|Waste,...,Emissions|VOC|Energy Sector,Emissions|VOC|Forest Burning,Emissions|VOC|Grassland Burning,Emissions|VOC|Industrial Sector,Emissions|VOC|International Shipping,Emissions|VOC|Peat Burning,Emissions|VOC|Residential Commercial Other,Emissions|VOC|Solvents Production and Application,Emissions|VOC|Transportation Sector,Emissions|VOC|Waste
TODO,SET,SET,SET,SET,SET,SET,SET,SET,SET,SET,...,SET,SET,SET,SET,SET,SET,SET,SET,SET,SET
UNITS,Mt BC/yr,Mt BC/yr,Mt BC/yr,Mt BC/yr,Mt BC/yr,Mt BC/yr,Mt BC/yr,Mt BC/yr,Mt BC/yr,Mt BC/yr,...,Mt VOC/yr,Mt VOC/yr,Mt VOC/yr,Mt VOC/yr,Mt VOC/yr,Mt VOC/yr,Mt VOC/yr,Mt VOC/yr,Mt VOC/yr,Mt VOC/yr
REGION,R5ASIA,R5ASIA,R5ASIA,R5ASIA,R5ASIA,R5ASIA,R5ASIA,R5ASIA,R5ASIA,R5ASIA,...,World,World,World,World,World,World,World,World,World,World
YEAR,,,,,,,,,,,,,,,,,,,,,
2015,2,8,7,5,6,1,5,1,8,8,...,2,4,10,4,8,7,1,4,7,0
2020,7,1,4,2,7,9,5,8,2,1,...,2,1,10,5,7,10,6,10,1,10
2030,3,9,6,1,7,4,5,4,2,1,...,8,10,1,5,5,6,10,1,2,1
2040,10,8,6,10,4,9,8,8,4,8,...,0,9,0,4,1,3,2,0,6,4
2050,7,2,9,8,3,0,4,2,6,6,...,7,0,10,2,9,0,2,8,0,8
2060,0,9,1,10,2,5,0,8,10,8,...,2,7,3,8,8,8,10,6,4,4


{'MODEL': 'MODEL-NAME-HYPHENS', 'SCENARIO': 'SCENARIO-NAME-HYPHENS'}

...

MODEL          NAME-MODEL-2                                          \
SCENARIO SCENARIO-A-B-CDE-2                                           
VARIABLE       Emissions|BC Emissions|BC|Agricultural Waste Burning   
TODO                    SET                                     SET   
UNITS              Mt BC/yr                                Mt BC/yr   
REGION               R5ASIA                                  R5ASIA   
YEAR                                                                  
2015                      9                                       2   
2020                      3                                       3   
2030                      7                                       2   
2040                     10                                       5   
2050                      1                                       7   
2060                      1                                       8   
2070                      9                                       9   
2080                      1                                      10   
2090                      6                                       1   
2100                      7                                       7   

MODEL                                                            \
SCENARIO                                                          
VARIABLE Emissions|BC|Energy Sector Emissions|BC|Forest Burning   
TODO                            SET                         SET   
UNITS                      Mt BC/yr                    Mt BC/yr   
REGION                       R5ASIA                      R5ASIA   
YEAR                                                              
2015                              6                           0   
2020                              6                           9   
2030                              9                           4   
2040                              1                           2   
2050                              7                           3   
2060                              8                           1   
2070                              3                           2   
2080                              0                           7   
2090                              5                           8   
2100                              4                           8   

MODEL                                                                   \
SCENARIO                                                                 
VARIABLE Emissions|BC|Grassland Burning Emissions|BC|Industrial Sector   
TODO                                SET                            SET   
UNITS                          Mt BC/yr                       Mt BC/yr   
REGION                           R5ASIA                         R5ASIA   
YEAR                                                                     
2015                                  3                             10   
2020                                  6                              2   
2030                                  3                             10   
2040                                  1                              4   
2050                                  7                              6   
2060                                  0                              5   
2070                                  4                              9   
2080                                  4                              4   
2090                                  7                              6   
2100                                  3                              3   

MODEL                                                                         \
SCENARIO                                                                       
VARIABLE Emissions|BC|Peat Burning Emissions|BC|Residential Commercial Other   
TODO                           SET                                       SET   
UNITS                     Mt BC/yr                                  Mt BC

VARIABLE,Emissions|BC,Emissions|BC|Agricultural Waste Burning,Emissions|BC|Energy Sector,Emissions|BC|Forest Burning,Emissions|BC|Grassland Burning,Emissions|BC|Industrial Sector,Emissions|BC|Peat Burning,Emissions|BC|Residential Commercial Other,Emissions|BC|Transportation Sector,Emissions|BC|Waste,...,Emissions|VOC|Energy Sector,Emissions|VOC|Forest Burning,Emissions|VOC|Grassland Burning,Emissions|VOC|Industrial Sector,Emissions|VOC|International Shipping,Emissions|VOC|Peat Burning,Emissions|VOC|Residential Commercial Other,Emissions|VOC|Solvents Production and Application,Emissions|VOC|Transportation Sector,Emissions|VOC|Waste
TODO,SET,SET,SET,SET,SET,SET,SET,SET,SET,SET,...,SET,SET,SET,SET,SET,SET,SET,SET,SET,SET
UNITS,Mt BC/yr,Mt BC/yr,Mt BC/yr,Mt BC/yr,Mt BC/yr,Mt BC/yr,Mt BC/yr,Mt BC/yr,Mt BC/yr,Mt BC/yr,...,Mt VOC/yr,Mt VOC/yr,Mt VOC/yr,Mt VOC/yr,Mt VOC/yr,Mt VOC/yr,Mt VOC/yr,Mt VOC/yr,Mt VOC/yr,Mt VOC/yr
REGION,R5ASIA,R5ASIA,R5ASIA,R5ASIA,R5ASIA,R5ASIA,R5ASIA,R5ASIA,R5ASIA,R5ASIA,...,World,World,World,World,World,World,World,World,World,World
YEAR,,,,,,,,,,,,,,,,,,,,,
2015,9,2,6,0,3,10,10,4,1,2,...,0,3,6,3,1,7,10,6,6,10
2020,3,3,6,9,6,2,2,3,10,4,...,2,6,5,10,3,7,2,3,7,2
2030,7,2,9,4,3,10,7,1,2,8,...,5,1,0,10,5,2,8,9,8,1
2040,10,5,1,2,1,4,9,9,0,4,...,2,4,1,8,0,1,9,2,0,6
2050,1,7,7,3,7,6,10,5,7,3,...,3,4,3,0,3,8,0,3,6,9
2060,1,8,8,1,0,5,3,4,3,9,...,9,10,4,6,2,3,0,3,3,4


{'MODEL': 'NAME-MODEL-2', 'SCENARIO': 'SCENARIO-NAME-HYPHENS'}

MODEL             NAME-MODEL-2                                          \
SCENARIO SCENARIO-NAME-HYPHENS                                           
VARIABLE          Emissions|BC Emissions|BC|Agricultural Waste Burning   
TODO                       SET                                     SET   
UNITS                 Mt BC/yr                                Mt BC/yr   
REGION                  R5ASIA                                  R5ASIA   
YEAR                                                                     
2015                         7                                       4   
2020                        10                                       9   
2030                        10                                       5   
2040                         6                                       8   
2050                         8                                       8   
2060                        10                                       9   
2070                         1                                       1   
2080                         4                                       1   
2090                         5                                       9   
2100                         0                                       8   

MODEL                                                            \
SCENARIO                                                          
VARIABLE Emissions|BC|Energy Sector Emissions|BC|Forest Burning   
TODO                            SET                         SET   
UNITS                      Mt BC/yr                    Mt BC/yr   
REGION                       R5ASIA                      R5ASIA   
YEAR                                                              
2015                              6                           4   
2020                              2                           9   
2030                              0                           5   
2040                              0                           3   
2050                              8                           5   
2060                              7                           0   
2070                              4                           1   
2080                              9                           8   
2090                              7                           9   
2100                              3                           7   

MODEL                                                                   \
SCENARIO                                                                 
VARIABLE Emissions|BC|Grassland Burning Emissions|BC|Industrial Sector   
TODO                                SET                            SET   
UNITS                          Mt BC/yr                       Mt BC/yr   
REGION                           R5ASIA                         R5ASIA   
YEAR                                                                     
2015                                  4                              4   
2020                                  0                              4   
2030                                  4                              9   
2040                                  7                              7   
2050                                  6                              1   
2060                                  0                              6   
2070                                  5                              8   
2080                                  6                              3   
2090                                 10                              1   
2100                                  2                              3   

MODEL                                                                         \
SCENARIO                                                                       
VARIABLE Emissions|BC|Peat Burning Emissions|BC|Residential Commercial Other   
TODO                           SET                                       SET   
UNITS                 

VARIABLE,Emissions|BC,Emissions|BC|Agricultural Waste Burning,Emissions|BC|Energy Sector,Emissions|BC|Forest Burning,Emissions|BC|Grassland Burning,Emissions|BC|Industrial Sector,Emissions|BC|Peat Burning,Emissions|BC|Residential Commercial Other,Emissions|BC|Transportation Sector,Emissions|BC|Waste,...,Emissions|VOC|Energy Sector,Emissions|VOC|Forest Burning,Emissions|VOC|Grassland Burning,Emissions|VOC|Industrial Sector,Emissions|VOC|International Shipping,Emissions|VOC|Peat Burning,Emissions|VOC|Residential Commercial Other,Emissions|VOC|Solvents Production and Application,Emissions|VOC|Transportation Sector,Emissions|VOC|Waste
TODO,SET,SET,SET,SET,SET,SET,SET,SET,SET,SET,...,SET,SET,SET,SET,SET,SET,SET,SET,SET,SET
UNITS,Mt BC/yr,Mt BC/yr,Mt BC/yr,Mt BC/yr,Mt BC/yr,Mt BC/yr,Mt BC/yr,Mt BC/yr,Mt BC/yr,Mt BC/yr,...,Mt VOC/yr,Mt VOC/yr,Mt VOC/yr,Mt VOC/yr,Mt VOC/yr,Mt VOC/yr,Mt VOC/yr,Mt VOC/yr,Mt VOC/yr,Mt VOC/yr
REGION,R5ASIA,R5ASIA,R5ASIA,R5ASIA,R5ASIA,R5ASIA,R5ASIA,R5ASIA,R5ASIA,R5ASIA,...,World,World,World,World,World,World,World,World,World,World
YEAR,,,,,,,,,,,,,,,,,,,,,
2015,7,4,6,4,4,4,6,4,3,5,...,5,2,8,5,2,1,0,8,2,6
2020,10,9,2,9,0,4,0,5,6,0,...,10,2,5,4,4,6,4,6,9,7
2030,10,5,0,5,4,9,6,1,7,0,...,4,7,6,5,2,1,1,4,7,5
2040,6,8,0,3,7,7,8,3,9,10,...,2,0,1,4,8,0,8,1,8,10
2050,8,8,8,5,6,1,10,7,5,7,...,4,1,0,7,1,10,4,5,4,2
2060,10,9,7,0,0,6,3,10,4,4,...,8,5,9,0,9,8,6,3,6,10
